<font size="5">Performance Measurement UltraTrail</font>

In this notebook the pretrained Akida model which was trained according to the UltraTrail experimental setup, was mapped to the NP of the development kit. Additionally, the energy consumption is reported. This code is inspired by
https://doc.brainchipinc.com/examples/general/plot_1_akidanet_imagenet.html#sphx-glr-examples-general-plot-1-akidanet-imagenet-py

In [1]:
import os
from os import listdir
from os.path import isdir, join
import pathlib

import akida
from akida import FullyConnected
from akida import evaluate_sparsity
import cnn2snn
from cnn2snn import check_model_compatibility
from cnn2snn import quantize
from cnn2snn import quantize_layer
from cnn2snn import convert

from math import ceil

from timeit import default_timer as timer
import numpy as np

In [2]:
# Load the quantized CNN

quantized_model = cnn2snn.load_quantized_model('final_quantized_ultratrail_model.h5')

In [3]:
#  Convert to an Akida model

akida_model = convert(quantized_model)
akida_model.summary()

                 Model Summary                 
_______________________________________________
Input shape   Output shape  Sequences  Layers
[40, 101, 1]  [1, 1, 12]    1          10    
_______________________________________________

                SW/conv_0-dense (Software)                 
___________________________________________________________
Layer (type)             Output shape   Kernel shape     
conv_0 (InputConv.)      [51, 20, 32]   (3, 3, 1, 32)    
___________________________________________________________
separable_1 (Sep.Conv.)  [51, 20, 32]   (3, 3, 32, 1)    
___________________________________________________________
                                        (1, 1, 32, 32)   
___________________________________________________________
separable_2 (Sep.Conv.)  [26, 10, 64]   (3, 3, 32, 1)    
___________________________________________________________
                                        (1, 1, 32, 64)   
_______________________________________________________

In [4]:
# Load the data set

feature_sets_filename = 'final1_stored_files_targets_int_normalized.npz'

feature_sets = np.load(feature_sets_filename)

print('Feature Sets: ', feature_sets.files)

Feature Sets:  ['x_train', 'y_train', 'x_val', 'y_val', 'x_test', 'y_test']


In [5]:
# Assign test set

x_test = feature_sets['x_test']
y_test = feature_sets['y_test']

In [6]:
# Reshape to fit dimensions

x_test = x_test.reshape(x_test.shape[0], 
                        x_test.shape[1], 
                        x_test.shape[2], 
                        1)

print(x_test.shape)

(4371, 40, 101, 1)


In [7]:
# Define device

devices = akida.devices()
print('Akida NP available :', devices[0])

Akida NP available : <akida.core.HardwareDevice object at 0xffff4afa9870>


In [8]:
# Print version of NP

print(devices[0].version)

BC.00.000.002


In [9]:
# Map Akida model to device

device = devices[0]

akida_model.map(device)

akida_model.summary()

                 Model Summary                 
_______________________________________________
Input shape   Output shape  Sequences  Layers
[40, 101, 1]  [1, 1, 12]    3          10    
_______________________________________________

     HW/conv_0-separable_7 (Hardware) - size: 204356 bytes     
_______________________________________________________________
Layer (type)             Output shape   Kernel shape      NPs
conv_0 (InputConv.)      [51, 20, 32]   (3, 3, 1, 32)     N/A
_______________________________________________________________
separable_1 (Sep.Conv.)  [51, 20, 32]   (3, 3, 32, 1)     2  
_______________________________________________________________
                                        (1, 1, 32, 32)       
_______________________________________________________________
separable_2 (Sep.Conv.)  [26, 10, 64]   (3, 3, 32, 1)     2  
_______________________________________________________________
                                        (1, 1, 32, 64)       
_______

In [45]:
# Enable power measurement

device.soc.power_measurement_enabled = True

# Send data for inference

start = timer()
print('Time before test: ',timer())
_ = akida_model.forward(x_test)
events = device.soc.power_meter.events
for i in range(30):
    print('Voltage: ',events()[i].voltage)
    print('At Timestamp: ', events()[i].ts)
print('Time after test: ',timer())
end = timer()

# Display floor current

floor_power = device.soc.power_meter.floor
print(f'Floor power: {floor_power:.2f} mW')

# Retrieve statistics

print(akida_model.statistics)



Time before test:  3551.562399509
Voltage:  893750
At Timestamp:  3551696
Voltage:  893750
At Timestamp:  3551697
Voltage:  893750
At Timestamp:  3551781
Voltage:  893750
At Timestamp:  3551937
Voltage:  893750
At Timestamp:  3551938
Voltage:  893750
At Timestamp:  3552035
Voltage:  893750
At Timestamp:  3552095
Voltage:  893750
At Timestamp:  3552156
Voltage:  893750
At Timestamp:  3552278
Voltage:  893750
At Timestamp:  3552308
Voltage:  893750
At Timestamp:  3552405
Voltage:  893750
At Timestamp:  3552460
Voltage:  893750
At Timestamp:  3552509
Voltage:  893750
At Timestamp:  3552628
Voltage:  893750
At Timestamp:  3552661
Voltage:  893750
At Timestamp:  3552733
Voltage:  893750
At Timestamp:  3552785
Voltage:  893750
At Timestamp:  3552861
Voltage:  893750
At Timestamp:  3552931
Voltage:  893750
At Timestamp:  3553009
Voltage:  893750
At Timestamp:  3553077
Voltage:  893750
At Timestamp:  3553145
Voltage:  893750
At Timestamp:  3553213
Voltage:  893750
At Timestamp:  3553280
Voltag

In [34]:
# Print inference

print(f'Akida inference {len(x_test)} MFCC took {end-start:.2f} s.\n')

Akida inference 4371 MFCC took 5.83 s.



<font size="5">Investigation on Power Consumption with different Quantization Bit-Widths</font>

In [35]:
# Quanitize to 1-bit activations

quantized1_model = quantize(quantized_model,
                           input_weight_quantization=8,
                           weight_quantization=4,
                           activ_quantization=1)
akida_model = convert(quantized1_model)
device = devices[0]

akida_model.map(device)

akida_model.summary()

                 Model Summary                 
_______________________________________________
Input shape   Output shape  Sequences  Layers
[40, 101, 1]  [1, 1, 12]    3          10    
_______________________________________________

     HW/conv_0-separable_7 (Hardware) - size: 204356 bytes     
_______________________________________________________________
Layer (type)             Output shape   Kernel shape      NPs
conv_0 (InputConv.)      [51, 20, 32]   (3, 3, 1, 32)     N/A
_______________________________________________________________
separable_1 (Sep.Conv.)  [51, 20, 32]   (3, 3, 32, 1)     2  
_______________________________________________________________
                                        (1, 1, 32, 32)       
_______________________________________________________________
separable_2 (Sep.Conv.)  [26, 10, 64]   (3, 3, 32, 1)     2  
_______________________________________________________________
                                        (1, 1, 32, 64)       
_______

In [36]:
# Source: https://doc.brainchipinc.com/examples/general/plot_1_akidanet_imagenet.html#sphx-glr-examples-general-plot-1-akidanet-imagenet-py
# Enable power measurement

device.soc.power_measurement_enabled = True

# Send data for inference

start = timer()
_ = akida_model.forward(x_test)
end = timer()

# Display floor current

floor_power = device.soc.power_meter.floor
print(f'Floor power: {floor_power:.2f} mW')

# Retrieve statistics

print(akida_model.statistics)

Floor power: 913.41 mW

Sequence HW/conv_0-separable_7
Average framerate = 3133.33 fps
Last inference power range (mW):  Avg 946.25 / Min 933.00 / Max 951.00 / Std 5.31 
Last inference energy consumed (mJ/frame): 0.30
Sequence SW/separable_8
Average framerate = 1167.78 fps
Last inference power range (mW):  Avg 916.96 / Min 916.00 / Max 935.00 / Std 3.67 
Last inference energy consumed (mJ/frame): 0.79
Sequence HW/dense
Average framerate = 6632.78 fps
Last inference power range (mW):  Avg 914.56 / Min 913.00 / Max 917.00 / Std 1.88 
Last inference energy consumed (mJ/frame): 0.14


In [37]:
print(f'Akida inference {len(x_test)} MFCC took {end-start:.2f} s.\n')

Akida inference 4371 MFCC took 5.88 s.



In [38]:
quantized1_model = quantize(quantized_model,
                           input_weight_quantization=8,
                           weight_quantization=2,
                           activ_quantization=1)
akida_model = convert(quantized1_model)
device = devices[0]

akida_model.map(device)

akida_model.summary()

                 Model Summary                 
_______________________________________________
Input shape   Output shape  Sequences  Layers
[40, 101, 1]  [1, 1, 12]    3          10    
_______________________________________________

     HW/conv_0-separable_7 (Hardware) - size: 110916 bytes     
_______________________________________________________________
Layer (type)             Output shape   Kernel shape      NPs
conv_0 (InputConv.)      [51, 20, 32]   (3, 3, 1, 32)     N/A
_______________________________________________________________
separable_1 (Sep.Conv.)  [51, 20, 32]   (3, 3, 32, 1)     2  
_______________________________________________________________
                                        (1, 1, 32, 32)       
_______________________________________________________________
separable_2 (Sep.Conv.)  [26, 10, 64]   (3, 3, 32, 1)     2  
_______________________________________________________________
                                        (1, 1, 32, 64)       
_______

In [39]:
#Source: https://doc.brainchipinc.com/examples/general/plot_1_akidanet_imagenet.html#sphx-glr-examples-general-plot-1-akidanet-imagenet-py
# Enable power measurement

device.soc.power_measurement_enabled = True

# Send data for inference

start = timer()
_ = akida_model.forward(x_test)
end = timer()

# Display floor current

floor_power = device.soc.power_meter.floor
print(f'Floor power: {floor_power:.2f} mW')

# Retrieve statistics

print(akida_model.statistics)

Floor power: 913.41 mW

Sequence HW/conv_0-separable_7
Average framerate = 2522.22 fps
Last inference power range (mW):  Avg 970.05 / Min 953.00 / Max 975.00 / Std 5.98 
Last inference energy consumed (mJ/frame): 0.38
Sequence SW/separable_8
Average framerate = 1165.60 fps
Last inference power range (mW):  Avg 914.21 / Min 913.00 / Max 940.00 / Std 5.22 
Last inference energy consumed (mJ/frame): 0.78
Sequence HW/dense
Average framerate = 6409.09 fps
Last inference power range (mW):  Avg 913.22 / Min 913.00 / Max 914.00 / Std 0.44 
Last inference energy consumed (mJ/frame): 0.14


In [40]:
print(f'Akida inference {len(x_test)} MFCC took {end-start:.2f} s.\n')

Akida inference 4371 MFCC took 6.23 s.

